# Perfect artifical data (sd = 0)

is not possible, currently.

# Noisy artificial data

Check that everything is working on perfectly reproducing artifical data (with slight noise).

In [10]:
import imp
import footprint
imp.reload(footprint)
from test import generate_large_data_set
import numpy as np


## set conditions for data set generation
rtot = 0.02
sd_bp = 0.1
sd_height = 50
number_of_peaks = 3
number_of_footprints = 0 #number_of_peaks - 
number_of_control_traces = 3
number_of_sample_traces = 0

peaks_bp = np.arange(1,number_of_peaks,1) 
footprinted_peaks_bp_and_kd = np.dstack((np.random.choice(number_of_peaks,size=number_of_footprints,replace=False) , np.random.random_sample((number_of_footprints))))[0]
print(footprinted_peaks_bp_and_kd)
lfree_concs = np.append(np.random.random_integers(0,0,number_of_control_traces -1),np.random.random_integers(1,15,number_of_sample_traces)) #
print(lfree_concs)

trace_list = generate_large_data_set.generate_data_set(rtot, peaks_bp, footprinted_peaks_bp_and_kd , lfree_concs, sd_bp, sd_height)

print(trace_list[0])
ref = footprint.generate_averaged_negative_control(trace_list)
#traces = trace_list[1:]


print(len(trace_list))

footprint.cluster_peaks(ref,trace_list)

print(ref)
print("")
print(trace_list[0])
print("")
print(trace_list[-1])

[]
[0 0]
{'peaks': [{'peak_height': 3824.0, 'size_bp': 1.0}, {'peak_height': 300.0, 'size_bp': 2.0}], 'Ltot_conc': 0.0, 'dye_color': 'B', 'file_name': '0', 'Rtot_conc': 0.02}
1
2
3
{'peaks': [{'averaged_peak_size_bp_n': 3, 'averaged_peak_height_n': 3, 'averaged_peak_size_bp_nm': 1.0, 'averaged_peak_height_nm': 1.0, 'peak_height': 3827.92790785838, 'cluster': 1, 'averaged_peak_height_sd': 4.6408462789843066, 'averaged_peak_size_bp_sd': 0.019417126008468372, 'size_bp': 1.0241305694248901}, {'averaged_peak_size_bp_n': 3, 'averaged_peak_height_n': 3, 'averaged_peak_size_bp_nm': 1.0, 'averaged_peak_height_nm': 1.0, 'peak_height': 284.2516721058871, 'cluster': 2, 'averaged_peak_height_sd': 19.800485623570815, 'averaged_peak_size_bp_sd': 0.056219636677356931, 'size_bp': 2.078156572641819}], 'Ltot_conc': 0.0, 'dye_color': 'B', 'file_name': 'averaged_negative_control', 'Rtot_conc': 0.0}

{'peaks': [{'peak_height': 3824.0, 'cluster': 1, 'size_bp': 1.0}, {'peak_height': 300.0, 'cluster': 2, 'size

In [11]:

storage = []

%matplotlib
import matplotlib.pyplot as plt

for trace in trace_list:
    xdata = []
    ydata = []
    for factor in np.linspace(start=0.5,stop=3,num=100):
        footprint.correct_peaks_with_factor(trace,factor)
        #for weight_small, weight_big in [(0,1),(1,0)]:
        #    for relative in [True, False]:
        #        print(trace.file_name, factor,weight_small,weight_big,relative)
        #        index = " ".join([str(i) for i in [trace.file_name, factor,weight_small,weight_big,relative]])
        #        storage.append({index : footprint.calculate_deviance_for_all_peaks(ref,trace,weight_small,weight_big,relative)})
        xdata.append(factor)
        ydata.append(footprint.calculate_deviance_for_all_peaks(trace_list[0],trace,1,1,False,0,2000))
        footprint.correct_peaks_with_factor(trace,1/factor)
    plt.plot(xdata,ydata)


Using matplotlib backend: MacOSX


In [ ]:
imp.reload(footprint)
for t in trace_list: footprint.add_fractional_occupancies(ref,t)
footprint.fit_data_determine_kd(ref,trace_list)